# archive to parquet

Convert a remote archive or csv file (or local file://), to parquet format

In [56]:
import mlrun
import os
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters


In [67]:
BASE_IMAGE         = 'yjbds/mlrun-files:latest'

CODE_BASE          = '/User/repos/functions/'
TARGET_PATH        = '/User/mlrun/airlines/dataset'

ARCHIVE_BIG        = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears_10.csv"
ARCHIVE           = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears.csv"
ARCHIVE_SMALL      = "https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv"

**For testing and development use ARCHIVE_SMALL:**

In [68]:
USE_ARCHIVE = ARCHIVE

In [69]:
FILE_NAME          = 'airlines.pqt'
KEY                = 'airlines'

# no need for this as the files contain a header:
HEADER = ['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime','CRSArrTime',
          'UniqueCarrier','FlightNum','TailNum','ActualElapsedTime','CRSElapsedTime','AirTime',
          'ArrDelay','DepDelay','Origin','Dest','Distance','TaxiIn','TaxiOut','Cancelled',
          'CancellationCode','Diverted','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay',
          'LateAircraftDelay']

In [70]:
os.makedirs(TARGET_PATH, exist_ok=True)

## load and configure function

**If run the first time, create the function:**

In [71]:
# load function from a local Python file
arctoparq = mlrun.code_to_function(
    filename=os.path.join(CODE_BASE, 'fileutils/arc_to_parquet', 'arc_to_parquet.py'), 
    kind='job')
arctoparq.build_config(base_image=BASE_IMAGE, commands=[])
yaml_name = os.path.join(CODE_BASE, 'fileutils/arc_to_parquet', 'arc_to_parquet.yaml')
arctoparq.export(yaml_name)

[mlrun] 2020-01-27 15:27:39,471 function spec saved to path: /User/repos/functions/fileutils/arc_to_parquet/arc_to_parquet.yaml


**otherwise load it:**

In [72]:
arctoparq = mlrun.import_function(
    os.path.join(CODE_BASE, 'fileutils/arc_to_parquet', 'arc_to_parquet.yaml')
).apply(mlrun.mount_v3io())

## deploy / build

The following triggers a build when run for the first time using specs found in the yaml file above.

In [73]:
arctoparq.deploy(skip_deployed=True, with_mlrun=False)

'ready'

In [ ]:
# create and run the task
arc_to_parq_task = mlrun.NewTask(
    'arc2parq', 
    handler='arc_to_parquet',  
    params={
        'target_path': TARGET_PATH,
        'name'       : FILE_NAME, 
        'key'        : KEY,
        'archive_url': USE_ARCHIVE,
        'dataset'    : True,
        'part_cols'  : ['Year', 'Month']
    })
# run
run = arctoparq.run(arc_to_parq_task)

[mlrun] 2020-01-27 15:27:41,838 starting run arc2parq uid=d3774f73733c4abeb45c9dd7a7a4862c  -> http://mlrun-api:8080
[mlrun] 2020-01-27 15:27:41,973 Job is running in the background, pod: arc2parq-6d86t


___

## tests

### single parquet file

run this only when `dataset=False`

In [40]:
import os
import numpy as np
import pandas as pd

In [41]:
# add more context tests
# convert these to real tests

In [42]:
assert KEY in run.outputs.keys(), f"mlrun.functions: key {KEY} not found in outputs"
assert os.path.isfile(TARGET_PATH+'/'+ FILE_NAME),  f"mlrun.functions: artifact source not found at {TARGET_PATH+'/'+ FILE_NAME}"

In [43]:
copied   = pd.read_parquet(TARGET_PATH+'/'+ FILE_NAME, engine="pyarrow")

In [44]:
copied.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed


In [45]:
copied.shape

(0, 31)

run this only when `dataset=True`

In [47]:
import pyarrow.parquet as pq
import pyarrow as pa

In [65]:
table = pq.read_table(TARGET_PATH)

ValueError: Schema in /User/mlrun/airlines/dataset/a3f9441653c14708ad69f63694749301.parquet was different. 
Year: int64
Month: int64
DayofMonth: int64
DayOfWeek: int64
DepTime: double
CRSDepTime: int64
ArrTime: double
CRSArrTime: int64
UniqueCarrier: string
FlightNum: int64
TailNum: double
ActualElapsedTime: double
CRSElapsedTime: int64
AirTime: double
ArrDelay: double
DepDelay: double
Origin: string
Dest: string
Distance: double
TaxiIn: double
TaxiOut: double
Cancelled: int64
CancellationCode: double
Diverted: int64
CarrierDelay: double
WeatherDelay: double
NASDelay: double
SecurityDelay: double
LateAircraftDelay: double
IsArrDelayed: string
IsDepDelayed: string
metadata
--------
OrderedDict([(b'pandas',
              b'{"index_columns": [{"kind": "range", "name": null, "start": '
              b'0, "stop": 10000, "step": 1}], "column_indexes": [{"name": n'
              b'ull, "field_name": null, "pandas_type": "unicode", "numpy_ty'
              b'pe": "object", "metadata": {"encodi

In [53]:
dataset = pq.ParquetDataset(TARGET_PATH)

Init signature:
pq.ParquetDataset(
    path_or_paths,
    filesystem=None,
    schema=None,
    metadata=None,
    split_row_groups=False,
    validate_schema=True,
    filters=None,
    metadata_nthreads=1,
    read_dictionary=None,
    memory_map=False,
    buffer_size=0,
)
Docstring:     
Encapsulates details of reading a complete Parquet dataset possibly
consisting of multiple files and partitions in subdirectories

Parameters
----------
path_or_paths : str or List[str]
    A directory name, single file name, or list of file names
filesystem : FileSystem, default None
    If nothing passed, paths assumed to be found in the local on-disk
    filesystem
metadata : pyarrow.parquet.FileMetaData
    Use metadata obtained elsewhere to validate file schemas
schema : pyarrow.parquet.Schema
    Use schema obtained elsewhere to validate file schemas. Alternative to
    metadata parameter
split_row_groups : boolean, default False
    Divide files into pieces for each row group in the file
val

In [51]:
dataset.read?

ValueError: Schema in /User/mlrun/airlines/dataset/a3f9441653c14708ad69f63694749301.parquet was different. 
Year: int64
Month: int64
DayofMonth: int64
DayOfWeek: int64
DepTime: double
CRSDepTime: int64
ArrTime: double
CRSArrTime: int64
UniqueCarrier: string
FlightNum: int64
TailNum: double
ActualElapsedTime: double
CRSElapsedTime: int64
AirTime: double
ArrDelay: double
DepDelay: double
Origin: string
Dest: string
Distance: double
TaxiIn: double
TaxiOut: double
Cancelled: int64
CancellationCode: double
Diverted: int64
CarrierDelay: double
WeatherDelay: double
NASDelay: double
SecurityDelay: double
LateAircraftDelay: double
IsArrDelayed: string
IsDepDelayed: string
metadata
--------
OrderedDict([(b'pandas',
              b'{"index_columns": [{"kind": "range", "name": null, "start": '
              b'0, "stop": 10000, "step": 1}], "column_indexes": [{"name": n'
              b'ull, "field_name": null, "pandas_type": "unicode", "numpy_ty'
              b'pe": "object", "metadata": {"encodi

## cleanup

In [ ]:
# os.remove(parquet_file_path)